<a href="https://colab.research.google.com/github/kwameasare/mcorp/blob/master/python_bbvi_ed2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import csv
import os
import time 

# Dependency imports
from absl import flags
# import sys
from six.moves import urllib
# import edward as ed
# from edward.models import Empirical, Gamma, Poisson
import tensorflow.compat.v1 as tf1
# tf1.disable_v2_behavior()
import tensorflow as tf
tf.executing_eagerly()
tf.enable_v2_behavior()

EAGER = False

import cProfile

import tensorflow_probability as tfp 
from tensorflow_probability import edward2 as ed
dir(ed)
import numpy as np

# np.set_printoptions(threshold=np.inf)


# flags.DEFINE_float("learning_rate",
#                    default=1e-4,
#                    help="Initial learning rate.")
# flags.DEFINE_integer("max_steps",
#                      default=200000,
#                      help="Number of training steps to run.")
# flags.DEFINE_list("layer_sizes",
#                   default=["100", "30", "15"],
#                   help="Comma-separated list denoting number of latent "
#                        "variables (stochastic units) per layer.")
# flags.DEFINE_float("shape",
#                    default=0.1,
#                    help="Shape hyperparameter for Gamma priors on latents.")
# flags.DEFINE_string("data_dir",
#                     default=os.path.join(os.getenv("TEST_TMPDIR", "/tmp"),
#                                          "deep_exponential_family/data"),
#                     help="Directory where data is stored (if using real data).")
# flags.DEFINE_string("model_dir",
#                     default=os.path.join(os.getenv("TEST_TMPDIR", "/tmp"),
#                                          "deep_exponential_family/"),
#                     help="Directory to put the model's fit.")
# flags.DEFINE_bool("fake_data",
#                   default=None,
#                   help="If true, uses fake data. Defaults to real data.")

FLAGS = flags.FLAGS


bag_of_words = np.random.poisson(5., size=[256, 32000])  # training data as matrix of counts
data_size, feature_size = bag_of_words.shape  # number of documents x words (vocabulary)
units = [100, 30, 15]  # number of stochastic units per layer
shape = 0.1  # Gamma shape parameter

w2 = ed.Gamma(0.1, 0.3, sample_shape=[units[2], units[1]])
w1 = ed.Gamma(0.1, 0.3, sample_shape=[units[1], units[0]])
w0 = ed.Gamma(0.1, 0.3, sample_shape=[units[0], feature_size])

z2 = ed.Gamma(0.1, 0.1, sample_shape=[data_size, units[2]])
z1 = ed.Gamma(shape, shape / tf.matmul(z2, w2))

z0 = ed.Gamma(shape, shape / tf.matmul(z1, w1))
x = ed.Gamma(0.1, tf.matmul(z0, w0))

print(bag_of_words)


def deep_exponential_family(data_size, feature_size, units, shape):
  """A multi-layered topic model over a documents-by-terms matrix."""
  w2 = ed.Gamma(0.1, 0.3, sample_shape=[units[2], units[1]], name="w2")
  w1 = ed.Gamma(0.1, 0.3, sample_shape=[units[1], units[0]], name="w1")
  w0 = ed.Gamma(0.1, 0.3, sample_shape=[units[0], feature_size], name="w0")

  z2 = ed.Gamma(0.1, 0.1, sample_shape=[data_size, units[2]], name="z2")
  z1 = ed.Gamma(shape, shape / tf.matmul(z2, w2), name="z1")
  z0 = ed.Gamma(shape, shape / tf.matmul(z1, w1), name="z0")
  x = ed.Poisson(tf.matmul(z0, w0), name="x")
  print(x)
  return x



# Generate from model: returns tf.Tensor of shape (data_size, feature_size).
x = deep_exponential_family(data_size = 256 , feature_size = 32000, units = [100, 30, 15], shape = 0.1)
# x.numpy()  # converts from eagerly executed tf.Tensor to np.ndarray
# with tf.Session() as sess: 
#   sess.run(x)

[[5 8 2 ... 5 5 4]
 [7 4 4 ... 3 4 5]
 [7 6 2 ... 7 4 7]
 ...
 [2 4 6 ... 2 9 6]
 [7 6 1 ... 2 6 3]
 [6 7 4 ... 4 7 4]]
RandomVariable("
[[5.4400e+02 4.5430e+03 1.0573e+04 ... 1.9490e+03 4.7251e+04 1.1225e+04]
 [7.5000e+01 1.0000e+00 1.9000e+02 ... 6.4000e+01 3.9000e+01 9.9000e+01]
 [2.3000e+01 4.0000e+01 3.7000e+01 ... 7.3000e+01 1.7900e+02 1.0300e+02]
 ...
 [8.0000e+01 9.0000e+00 2.6000e+01 ... 1.6000e+01 1.4800e+02 8.6000e+01]
 [3.6000e+01 3.8700e+02 7.0440e+04 ... 7.1000e+02 3.0200e+02 7.2545e+04]
 [1.7210e+03 5.1000e+02 4.8800e+02 ... 2.2600e+02 2.7900e+02 1.6300e+03]]", shape=(256, 32000), dtype=float32, device=/job:localhost/replica:0/task:0/device:CPU:0)


In [0]:
def trainable_positive_deterministic(shape, min_loc=1e-3, name=None):
  """Learnable Deterministic distribution over positive reals."""
  with tf.compat.v1.variable_scope(
      None, default_name="trainable_positive_deterministic"):
    unconstrained_loc = tf.compat.v1.get_variable("unconstrained_loc", shape)
    loc = tf.maximum(tf.nn.softplus(unconstrained_loc), min_loc)
    rv = ed.Deterministic(loc=loc, name=name)

    print(rv)
    return rv

def trainable_gamma(shape, min_concentration=1e-3, min_scale=1e-5, name=None):
  """Learnable Gamma via concentration and scale parameterization."""
  with tf.compat.v1.variable_scope(None, default_name="trainable_gamma"):
    unconstrained_concentration = tf.compat.v1.get_variable(
        "unconstrained_concentration",
        shape,
        initializer=tf.compat.v1.initializers.random_normal(
            mean=0.5, stddev=0.1))
    unconstrained_scale = tf.compat.v1.get_variable(
        "unconstrained_scale",
        shape,
        initializer=tf.compat.v1.initializers.random_normal(stddev=0.1))
    concentration = tf.maximum(tf.nn.softplus(unconstrained_concentration),
                               min_concentration)
    rate = tf.maximum(1. / tf.nn.softplus(unconstrained_scale), 1. / min_scale)
    rv = ed.Gamma(concentration=concentration, rate=rate, name=name)
    print(rv)
    return rv



eg = trainable_positive_deterministic(shape = (3, 6), min_loc=1e-3, name = "Dave")
eg2 = trainable_gamma(shape = (3, 6), min_concentration=1e-3, min_scale=1e-3, name = "Dave2")

RandomVariable("
[[0.4286835  0.4863159  0.76186407 0.9298632  0.53964955 0.44855154]
 [0.5355686  0.9443109  0.57048243 0.77937424 0.66257924 0.4894936 ]
 [0.78970104 0.54046637 0.9839853  0.5362346  0.5936479  0.38427094]]", shape=(3, 6), dtype=float32, device=/job:localhost/replica:0/task:0/device:CPU:0)
RandomVariable("
[[2.1674184e-03 5.6429963e-05 8.8354101e-04 6.9050246e-04 6.5078365e-04
  2.8824492e-04]
 [6.1750755e-04 5.3683505e-04 9.4312170e-05 2.5020866e-04 7.7176787e-04
  8.6535246e-04]
 [1.8631553e-04 1.2163562e-04 1.0036634e-03 2.3673593e-04 1.6675701e-03
  6.9796115e-05]]", shape=(3, 6), dtype=float32, device=/job:localhost/replica:0/task:0/device:CPU:0)


In [0]:
bag_of_words = np.random.poisson(5., size=[256, 32000])  # training data as matrix of counts
data_size, feature_size = bag_of_words.shape  # number of documents x words (vocabulary)
units = [100, 30, 15]  # number of stochastic units per layer
shape = 0.1  # Gamma shape parameter

print(bag_of_words.shape)

def deep_exponential_family_variational(data_size, feature_size, units):
  """Posterior approx. for deep exponential family p(w{0,1,2}, z{1,2,3} | x)."""
  qw2 = trainable_positive_deterministic([units[2], units[1]], name="qw2")
  qw1 = trainable_positive_deterministic([units[1], units[0]], name="qw1")
  qw0 = trainable_positive_deterministic([units[0], feature_size], name="qw0")
  qz2 = trainable_gamma([data_size, units[2]], name="qz2")
  qz1 = trainable_gamma([data_size, units[1]], name="qz1")
  qz0 = trainable_gamma([data_size, units[0]], name="qz0")
  
  return qw2, qw1, qw0, qz2, qz1, qz0


eg = deep_exponential_family_variational(data_size = (256, 32000), feature_size = (256, 32000), units = [100, 30, 15])


(256, 32000)
RandomVariable("
[[0.6255446  0.8114654  0.7891374  0.8886018  0.5394876  0.8364475
  0.8828017  0.71148735 0.55205303 0.57565504 0.73650116 0.5495552
  0.76067954 0.75581944 0.6400035  0.62803847 0.5910707  0.7573783
  0.560536   0.67564374 0.6093487  0.83882165 0.5412694  0.74518883
  0.7932688  0.7861298  0.539871   0.64821166 0.7191437  0.8682095 ]
 [0.66428435 0.7965848  0.5892857  0.82839143 0.59894127 0.7028111
  0.55705374 0.55072    0.8576871  0.69677234 0.8144127  0.7283403
  0.7874715  0.61951053 0.7539011  0.5319842  0.7861264  0.8289107
  0.5336695  0.5992313  0.8432434  0.53485703 0.8579883  0.56902957
  0.72489285 0.76901084 0.6550898  0.64060795 0.5540998  0.754682  ]
 [0.6768164  0.7588457  0.63601846 0.72876585 0.7111478  0.6830122
  0.6745105  0.55184966 0.7736014  0.6674448  0.60791355 0.6885026
  0.72230566 0.8266524  0.7298543  0.64486265 0.7808952  0.65651625
  0.79968935 0.6961094  0.73536354 0.8331728  0.58277434 0.69913733
  0.56186104 0.81925327 

TypeError: ignored

In [0]:
def load_nips2011_papers(path):
  """Loads NIPS 2011 conference papers.
  The NIPS 1987-2015 data set is in the form of a 11,463 x 5,812 matrix of
  per-paper word counts, containing 11,463 words and 5,811 NIPS conference
  papers (Perrone et al., 2016). We subset to papers in 2011 and words appearing
  in at least two documents and having a total word count of at least 10.
  Built from the Observations Python package.
  Args:
    path: str.
      Path to directory which either stores file or otherwise file will
      be downloaded and extracted there. Filename is `NIPS_1987-2015.csv`.
  Returns:
    bag_of_words: np.ndarray of shape [num_documents, num_words]. Each element
      denotes the number of occurrences of a specific word in a specific
      document.
    words: List of strings, denoting the words for `bag_of_words`'s columns.
  """
  path = os.path.expanduser(path)
  filename = "NIPS_1987-2015.csv"
  filepath = os.path.join(path, filename)
  if not os.path.exists(filepath):
    url = ("https://archive.ics.uci.edu/ml/machine-learning-databases/"
           "00371/NIPS_1987-2015.csv")
    if not tf.io.gfile.exists(path):
      tf.io.gfile.makedirs(path)
    print("Downloading %s to %s" % (url, filepath))
    urllib.request.urlretrieve(url, filepath)

  with open(filepath) as f:
    iterator = csv.reader(f)
    documents = next(iterator)[1:]
    words = []
    x_train = []
    for row in iterator:
      words.append(row[0])
      x_train.append(row[1:])

  x_train = np.array(x_train, dtype=np.int)

  # Subset to documents in 2011 and words appearing in at least two documents
  # and have a total word count of at least 10.
  doc_idx = [i for i, document in enumerate(documents)
             if document.startswith("2011")]
  documents = [documents[doc] for doc in doc_idx]
  x_train = x_train[:, doc_idx]
  word_idx = np.logical_and(np.sum(x_train != 0, 1) >= 2,
                            np.sum(x_train, 1) >= 10)
  words = [word for word, idx in zip(words, word_idx) if idx]
  bag_of_words = x_train[word_idx, :].T
  print(bag_of_words)
  print(words)
  
  print(x_train)
  
  return bag_of_words, words




In [0]:
eg = load_nips2011_papers("tmp")

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 3]
 [0 0 0 ... 0 0 0]]
['abc', 'abernethy', 'abilities', 'ability', 'ables', 'abs', 'absence', 'absent', 'absolute', 'abstract', 'abstraction', 'abstractions', 'abuse', 'academic', 'academy', 'accelerate', 'accelerated', 'acceleration', 'accept', 'acceptance', 'accepted', 'accepting', 'access', 'accesses', 'accommodate', 'accomplish', 'accomplished', 'accord', 'accordance', 'according', 'accordingly', 'account', 'accounted', 'accounting', 'accounts', 'accumulated', 'accumulation', 'accuracies', 'accuracy', 'accurate', 'accurately', 'achievable', 'achieve', 'achieved', 'achieves', 'achieving', 'acknowledge', 'acknowledgement', 'acknowledgements', 'acknowledges', 'acknowledgments', 'acl', 'acm', 'acoustics', 'acquire', 'acquired', 'acquisition', 'across', 'act', 'acting', 'action', 'actions', 'activated', 'activation', 'activations', 'active', 'actively', 'activities', 'activity', 'actor', 'a

In [0]:
total_count = np.sum(bag_of_words)
bag_of_words = tf.cast(bag_of_words, dtype=tf.float32)
data_size, feature_size = bag_of_words.shape

  # Compute expected log-likelihood. First, sample from the variational
  # distribution; second, compute the log-likelihood given the sample.
qw2, qw1, qw0, qz2, qz1, qz0 = deep_exponential_family_variational(
      data_size,
      feature_size,
      [100, 30, 15])


RandomVariable("
[[0.76638013 0.72634184 0.7091819  0.54773325 0.85444665 0.6916571
  0.5554572  0.8024089  0.61525583 0.6938492  0.7276032  0.59040785
  0.8366246  0.8037608  0.80762523 0.8003547  0.55389655 0.591243
  0.88974637 0.5595267  0.87698805 0.77876514 0.5947429  0.7203584
  0.8795036  0.5671319  0.67417073 0.75732875 0.55896056 0.5371249 ]
 [0.5504246  0.56875604 0.5691553  0.83667207 0.55526286 0.6242367
  0.75939953 0.8617787  0.7627822  0.5386111  0.62205166 0.5339345
  0.6016416  0.61345327 0.71098673 0.8518887  0.7600916  0.7139102
  0.79141295 0.8593241  0.76448977 0.8064695  0.64174527 0.65692806
  0.61968917 0.7453042  0.60552514 0.7882108  0.59880996 0.7486093 ]
 [0.81350017 0.7006977  0.7461702  0.7829429  0.728034   0.6492633
  0.8789792  0.7500843  0.8281819  0.5775149  0.7206584  0.6177919
  0.8413499  0.7966154  0.5536822  0.551145   0.6457198  0.7628359
  0.8899099  0.6256598  0.7431389  0.55260706 0.65095013 0.73998994
  0.56029296 0.8116073  0.535645   0.69

In [0]:
with ed.tape() as model_tape:
    with ed.interception(ed.make_value_setter(w2=qw2, w1=qw1, w0=qw0,
                                              z2=qz2, z1=qz1, z0=qz0)):
      posterior_predictive = deep_exponential_family(data_size,
                                                     feature_size,
                                                     [100, 30, 15],
                                                     0.1)

RandomVariable("
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]", shape=(256, 32000), dtype=float32, device=/job:localhost/replica:0/task:0/device:CPU:0)


In [0]:
log_likelihood = posterior_predictive.distribution.log_prob(bag_of_words)
log_likelihood = tf.reduce_sum(input_tensor=log_likelihood)
tf.compat.v1.summary.scalar("log_likelihood", log_likelihood)

<tf.Tensor: id=5596, shape=(), dtype=string, numpy=b'\n\x15\n\x0elog_likelihood\x15\x1aI\xa3\xcd'>

In [0]:
 # Compute analytic KL-divergence between variational and prior distributions.
  kl = 0.
  for rv_name, variational_rv in [("z0", qz0), ("z1", qz1), ("z2", qz2),
                                  ("w0", qw0), ("w1", qw1), ("w2", qw2)]:
    kl += tf.reduce_sum(
        input_tensor=variational_rv.distribution.kl_divergence(
            model_tape[rv_name].distribution))

  tf.compat.v1.summary.scalar("kl", kl)

<tf.Tensor: id=5704, shape=(), dtype=string, numpy=b'\n\t\n\x02kl\x15\xc5\x95\xddJ'>

In [0]:
elbo = log_likelihood - kl
tf.compat.v1.summary.scalar("elbo", elbo)
optimizer = tf.compat.v1.train.AdamOptimizer(1e-4)

train_op = optimizer.minimize(-elbo)
# sess = tf.compat.v1.Session()


RuntimeError: ignored

In [0]:



# def main(argv):
#   del argv  # unused
#   FLAGS.layer_sizes = [int(layer_size) for layer_size in FLAGS.layer_sizes]
#   if len(FLAGS.layer_sizes) != 3:
#     raise NotImplementedError("Specifying fewer or more than 3 layers is not "
#                               "currently available.")
#   if tf.io.gfile.exists(FLAGS.model_dir):
#     tf.compat.v1.logging.warning(
#         "Warning: deleting old log directory at {}".format(FLAGS.model_dir))
#     tf.io.gfile.rmtree(FLAGS.model_dir)
#   tf.io.gfile.makedirs(FLAGS.model_dir)

#   if FLAGS.fake_data:
#     bag_of_words = np.random.poisson(1., size=[10, 25])
#     words = [str(i) for i in range(25)]
#   else:
#     bag_of_words, words = load_nips2011_papers(FLAGS.data_dir)

#   total_count = np.sum(bag_of_words)
#   bag_of_words = tf.cast(bag_of_words, dtype=tf.float32)
#   data_size, feature_size = bag_of_words.shape

#   # Compute expected log-likelihood. First, sample from the variational
#   # distribution; second, compute the log-likelihood given the sample.
#   qw2, qw1, qw0, qz2, qz1, qz0 = deep_exponential_family_variational(
#       data_size,
#       feature_size,
#       FLAGS.layer_sizes)

#   with ed.tape() as model_tape:
#     with ed.interception(ed.make_value_setter(w2=qw2, w1=qw1, w0=qw0,
#                                               z2=qz2, z1=qz1, z0=qz0)):
#       posterior_predictive = deep_exponential_family(data_size,
#                                                      feature_size,
#                                                      FLAGS.layer_sizes,
#                                                      FLAGS.shape)

#   log_likelihood = posterior_predictive.distribution.log_prob(bag_of_words)
#   log_likelihood = tf.reduce_sum(input_tensor=log_likelihood)
#   tf.compat.v1.summary.scalar("log_likelihood", log_likelihood)

#   # Compute analytic KL-divergence between variational and prior distributions.
#   kl = 0.
#   for rv_name, variational_rv in [("z0", qz0), ("z1", qz1), ("z2", qz2),
#                                   ("w0", qw0), ("w1", qw1), ("w2", qw2)]:
#     kl += tf.reduce_sum(
#         input_tensor=variational_rv.distribution.kl_divergence(
#             model_tape[rv_name].distribution))

#   tf.compat.v1.summary.scalar("kl", kl)

#   elbo = log_likelihood - kl
#   tf.compat.v1.summary.scalar("elbo", elbo)
#   optimizer = tf.compat.v1.train.AdamOptimizer(FLAGS.learning_rate)
#   train_op = optimizer.minimize(-elbo)

#   sess = tf.compat.v1.Session()
#   summary = tf.compat.v1.summary.merge_all()
#   summary_writer = tf.compat.v1.summary.FileWriter(FLAGS.model_dir, sess.graph)
#   start_time = time.time()

#   sess.run(tf.compat.v1.global_variables_initializer())
#   for step in range(FLAGS.max_steps):
#     start_time = time.time()
#     _, elbo_value = sess.run([train_op, elbo])
#     if step % 500 == 0:
#       duration = time.time() - start_time
#       print("Step: {:>3d} Loss: {:.3f} ({:.3f} sec)".format(
#           step, elbo_value, duration))
#       summary_str = sess.run(summary)
#       summary_writer.add_summary(summary_str, step)
#       summary_writer.flush()

#       # Compute perplexity of the full data set. The model's negative
#       # log-likelihood of data is upper bounded by the variational objective.
#       negative_log_likelihood = -elbo_value
#       perplexity = np.exp(negative_log_likelihood / total_count)
#       print("Negative log-likelihood <= {:0.3f}".format(
#           negative_log_likelihood))
#       print("Perplexity <= {:0.3f}".format(perplexity))

#       # Print top 10 words for first 10 topics.
#       qw0_values = sess.run(qw0)
#       for k in range(min(10, FLAGS.layer_sizes[-1])):
#         top_words_idx = qw0_values[k, :].argsort()[-10:][::-1]
#         top_words = " ".join([words[i] for i in top_words_idx])
#         print("Topic {}: {}".format(k, top_words))

# if __name__ == "__main__":
#   tf.compat.v1.app.run()

In [0]:
# def build_trainable_positive_pointmass(shape, name=None):
#   """Builds point mass r.v. over positive reals and its parameters."""
#   mean = tf.Variable(tf.random.normal(shape))
#   print(mean)
#   def positive_pointmass():
#     return tf.nn.softplus(mean)
  
#   return positive_pointmass, [mean]

# def build_trainable_gamma(shape, name=None):
#   """Builds Gamma random variable and its parameters."""
#   shape_param = tf.Variable(tf.random.normal(shape))
#   scale_param = tf.Variable(tf.random.normal(shape))
#   def gamma():
#     return ed.Gamma(tf.nn.softplus(shape_param), 1./tf.nn.softplus(scale_param), name=name)
#   return gamma, [shape_param, scale_param]

# def build_deep_exponential_family_variational():
#   """Builds posterior approximation q(w{0,1,2}, z{1,2,3} | x) and parameters."""
#   QW2, qw2_params = build_trainable_positive_pointmass(w2.shape, name="qw2")
#   QW1, qw1_params = build_trainable_positive_pointmass(w1.shape, name="qw1")
#   QW0, qw0_params = build_trainable_positive_pointmass(w0.shape, name="qw0")
#   QZ2, qz2_params = build_trainable_gamma(z2.shape, name="qz2")
#   QZ1, qz1_params = build_trainable_gamma(z1.shape, name="qz1")
#   QZ0, qz0_params = build_trainable_gamma(z0.shape, name="qz0")
#   parameters = (qw2_params + qw1_params + qw0_params +
#                 qz2_params + qz1_params + qz0_params)
#   def deep_exponential_family_variational():
#     return QW2(), QW1(), QW0(), QZ2(), QZ1(), QZ0()
#   return deep_exponential_family_variational, parameters

# x = build_trainable_positive_pointmass((3,6))

In [0]:
# max_steps = 10000  # number of training iterations
# model_dir = None  # directory for model checkpoints

# writer = tf.compat.v1.summary.create_file_writer("tmp")
# [deep_exponential_family_variational,
#   trainable_variables] = build_deep_exponential_family_variational()

# @tf.function
# def train_step(bag_of_words, step):
#   with tf.GradientTape() as tape:
#     # Compute expected log-likelihood. First, sample from the variational
#     # distribution; second, compute the log-likelihood given the sample.
#     qw2, qw1, qw0, qz2, qz1, qz0 = deep_exponential_family_variational()

#     # Compute forward pass of model, setting value of the priors to the
#     # approximate posterior samples. We also record the forward pass' execution
#     # via ed.tape().
#     with ed.tape() as model_tape:
#       with ed.condition(w2=qw2, w1=qw1, w0=qw0,
#                         z2=qz2, z1=qz1, z0=qz0):
#         posterior_predictive = deep_exponential_family(data_size, feature_size, units, shape)

#     log_likelihood = posterior_predictive.distribution.log_prob(bag_of_words)

#     # Compute analytic KL-divergence between variational and prior distributions.
#     kl = 0.
#     for rv_name, variational_rv in [("z0", qz0), ("z1", qz1), ("z2", qz2),
#                                     ("w0", qw0), ("w1", qw1), ("w2", qw2)]:
#       kl += tf.reduce_sum(variational_rv.distribution.kl_divergence(
#           model_tape[rv_name].distribution))

#     elbo = tf.reduce_mean(log_likelihood - kl)
#     with writer.default():
#       tf.summary.scalar("elbo", elbo, step=step)
#       loss = -elbo
#   optimizer = tf.keras.optimizers.Adam(1e-3)
#   gradients = tape.gradient(loss, trainable_variables)
#   optimizer.apply_gradients(zip(gradients, trainable_variables))
#   return loss


#   for step in range(max_steps):
#     start_time = time.time()
#     bag_of_words = next(train_data)
#     loss = train_step(bag_of_words, step)
#     if step % 500 == 0:
#       writer.flush()
#       duration = time.time() - start_time
#       print("Step: {:>3d} Loss: {:.3f} ({:.3f} sec)".format(
#           step, elbo_value, duration))
      
# ter = train_step(bag_of_words, 0.01)